# Analysis of Belle2 Data

In [1]:
import os, sys, pprint
from root_pandas import read_root
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

Welcome to JupyROOT 6.16/00


/home/philipp/anaconda3/envs/BelleAnalysis/lib/python3.6/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.16.4 is currently installed but you installed root_numpy against numpy 1.9.3. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [2]:
!ls -l DATA

total 56200
-rwxr--r-- 1 philipp philipp   584128 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_ccbar_0.root
-rwxr--r-- 1 philipp philipp   117621 May 14 14:58 B0_etapr-eta-gg2pi_KS-pi+pi-_output_charged_17.root
-rwxr--r-- 1 philipp philipp   155084 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_0.root
-rwxr--r-- 1 philipp philipp   177149 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_1.root
-rwxr--r-- 1 philipp philipp   183977 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_2.root
-rwxr--r-- 1 philipp philipp   176779 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_3.root
-rwxr--r-- 1 philipp philipp   185028 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_4.root
-rwxr--r-- 1 philipp philipp   141664 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_5.root
-rwxr--r-- 1 philipp philipp   192110 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_6.root
-rwxr--r-- 1 philipp philipp   179731 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_7.root
-rwxr--r-

### Reading and Prepairing Data

In [7]:
Signal = read_root('DATA/B0_etapr-eta-gg2pi_KS-pi+pi-_output_sgnMC9_BGx1_0.root', 'B0')
DD = read_root('DATA/B0_etapr-eta-gg2pi_KS-pi+pi-_output_ddbar_0.root', 'B0')

In [15]:
Masses = ['B0_M', 'B0_ErrM', 'B0_SigM', 'B0_K_S0_M', 'B0_K_S0_ErrM', 'B0_K_S0_SigM', 'B0_etap_M', 'B0_etap_ErrM', 
          'B0_etap_SigM', 'B0_etap_eta_M', 'B0_etap_eta_ErrM', 'B0_etap_eta_SigM']
Kinetics = ['B0_P', 'B0_P4', 'B0_deltae', 'B0_mbc', 'B0_etap_P', 'B0_etap_P4', 
            'B0_etap_eta_P', 'B0_etap_eta_P4', 'B0_etap_eta_gamma0_P', 'B0_etap_eta_gamma0_P4',
            'B0_etap_eta_gamma1_P', 'B0_etap_eta_gamma1_P4', 'B0_etap_pi0_P', 'B0_etap_pi0_P4',
            'B0_etap_pi1_P', 'B0_etap_pi1_P4', 'B0_K_S0_P', 'B0_K_S0_P4']
DecayAngles = ['B0_decayAngle__bo0__bc', 'B0_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo0__bc',
               'B0_etap_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo2__bc']
Positions = ['B0_X', 'B0_ErrX', 'B0_Y', 'B0_ErrY', 'B0_Z', 'B0_ErrZ', 'B0_Rho', 'B0_VtxPvalue',
             'B0_VtxProdCov', 
             'B0_etap_X', 'B0_etap_ErrX', 'B0_etap_Y', 'B0_etap_ErrY', 
             'B0_etap_Z', 'B0_etap_ErrZ', 'B0_etap_Rho', 'B0_etap_VtxPvalue', 'B0_etap_VtxProd', 
             'B0_etap_VtxProdCov', 
             'B0_etap_eta_X', 'B0_etap_eta_ErrX', 'B0_etap_eta_Y',
             'B0_etap_eta_ErrY', 'B0_etap_eta_Z', 'B0_etap_eta_ErrZ', 'B0_etap_eta_Rho',
             'B0_etap_eta_VtxPvalue', 'B0_etap_eta_VtxProd', 'B0_etap_VtxProdCov', 
             'B0_etap_pi0_X', 'B0_etap_pi0_ErrX', 'B0_etap_pi0_Y', 'B0_etap_pi0_ErrY', 
             'B0_etap_pi0_Z', 'B0_etap_pi0_ErrZ', 'B0_etap_pi0_Rho', 'B0_etap_pi0_VtxPvalue', 
             'B0_etap_pi0_VtxProd', 'B0_etap_pi0_VtxProdCov',
             'B0_etap_pi1_X', 'B0_etap_pi1_ErrX', 'B0_etap_pi1_Y', 'B0_etap_pi1_ErrY', 
             'B0_etap_pi1_Z', 'B0_etap_pi1_ErrZ', 'B0_etap_pi1_Rho', 'B0_etap_pi1_VtxPvalue', 
             'B0_etap_pi1_VtxProd', 'B0_etap_pi1_VtxProdCov',
             'B0_K_S0_X', 'B0_K_S0_ErrX', 'B0_K_S0_Y', 'B0_K_S0_ErrY', 'B0_K_S0_Z',
             'B0_K_S0_ErrZ', 'B0_K_S0_Rho', 'B0_K_S0_VtxPvalue', 'B0_K_S0_VtxProdCov',
             'B0_cosAngleBetweenMomentumAndVertexVector', 'B0_distance', 'B0_significanceOfDistance',
             'B0_dr', 'B0_etap_pi0_dr', 'B0_etap_pi1_dr', 'B0_K_S0_dr']
Continuum_Supression = ['B0_CSMVA', 'B0_TrCSMVA']

In [18]:
DD[Kinetics]

,B0_P,B0_P4,B0_deltae,B0_mbc,B0_etap_P,B0_etap_P4,B0_etap_eta_P,B0_etap_eta_P4,B0_etap_eta_gamma0_P,B0_etap_eta_gamma0_P4,B0_etap_eta_gamma1_P,B0_etap_eta_gamma1_P4,B0_etap_pi0_P,B0_etap_pi0_P4,B0_etap_pi1_P,B0_etap_pi1_P4,B0_K_S0_P,B0_K_S0_P4
0,1.794983,"[0.540331, -0.40538287, 1.6630305, 5.6550207]",0.098378,5.263727,2.524425,"[-1.3704708, -2.1030047, 0.2681421, 2.6998782]",1.385079,"[-0.7340187, -1.1670384, 0.13297606, 1.4902903]",1.430163,"[-0.697255, -1.2461032, 0.080170296, 1.4301625]",0.122157,"[-0.068412445, 0.07038198, 0.07272185, 0.12215...",0.625619,"[-0.33160976, -0.5302377, 0.016805138, 0.64099...",0.299618,"[-0.21416827, -0.16801526, 0.12519379, 0.3305306]",2.911837,"[1.9108018, 1.6976218, 1.3948884, 2.9551423]"
1,1.694960,"[0.53885865, -0.3337717, 1.571979, 5.592729]",0.059453,5.270030,2.457671,"[-1.3719431, -2.0313935, 0.17709063, 2.6375868]",1.411758,"[-0.77167135, -1.1817852, 0.031099694, 1.5146725]",0.128521,"[-0.10158464, 0.054047942, -0.05724437, 0.1285...",1.430163,"[-0.697255, -1.2461032, 0.080170296, 1.4301625]",0.625619,"[-0.33160976, -0.5302377, 0.016805138, 0.64099...",0.299618,"[-0.21416827, -0.16801526, 0.12519379, 0.3305306]",2.911837,"[1.9108018, 1.6976218, 1.3948884, 2.9551423]"
2,1.795601,"[-0.3118448, 0.53493774, 1.6854602, 5.532878]",0.001732,5.231656,2.879012,"[-0.16300061, -1.8480448, 2.2015615, 3.0359828]",1.016235,"[0.012947869, -0.7146201, 0.7224152, 1.1546751]",0.309634,"[0.19088916, -0.24181731, 0.03096981, 0.30963433]",0.879674,"[-0.17030486, -0.495503, 0.7066107, 0.8796736]",1.255040,"[-0.06622166, -0.7582713, 0.99788004, 1.2627767]",0.508972,"[-0.061124567, -0.33585027, 0.37751955, 0.5277...",2.442769,"[-0.14884418, 2.3829825, -0.51610124, 2.4968948]"
3,2.051954,"[-0.17089856, -0.449695, 1.9947643, 5.461962]",-0.165816,5.229654,2.194316,"[1.9346491, 0.97312725, 0.35380504, 2.3964388]",1.314448,"[1.0602516, 0.7618655, 0.15231554, 1.4240488]",0.064875,"[0.0061204717, -0.050662644, -0.040057734, 0.0...",1.339787,"[1.0457778, 0.8140378, 0.19677824, 1.3397875]",0.717906,"[0.64579284, 0.19176148, 0.24813029, 0.7313476]",0.373625,"[0.33402526, 0.16316462, -0.037420493, 0.39884...",3.024980,"[-2.1055477, -1.4228222, 1.6409593, 3.0655231]"
4,1.482453,"[0.000104904175, -0.19658506, 1.4693611, 5.675...",0.197487,5.280221,2.761216,"[2.3077564, -1.3613918, 0.6672244, 2.9231334]",2.384579,"[1.995637, -1.1566801, 0.60476536, 2.4467053]",0.081359,"[0.05452697, 0.05961913, 0.009575924, 0.08135926]",2.369409,"[1.9441217, -1.2162682, 0.59596944, 2.3694086]",0.241867,"[0.1986824, -0.12726103, 0.053192925, 0.27924788]",0.136738,"[0.112749, -0.07686382, 0.00876638, 0.19538952]",2.706558,"[-2.3076515, 1.1648067, 0.8021367, 2.7519636]"
5,1.508138,"[0.46764934, -0.16297472, 1.4245077, 5.531454]",0.040591,5.281233,3.088949,"[-0.6030152, 1.3188246, 2.7273948, 3.2342074]",1.266983,"[-0.24763158, 0.5152959, 1.1306609, 1.3803579]",0.074785,"[0.07424237, -0.0025563347, -0.008620536, 0.07...",1.285633,"[-0.32879743, 0.5141967, 1.1315238, 1.2856331]",1.503111,"[-0.21778314, 0.6129132, 1.3550829, 1.5095766]",0.559694,"[-0.16595729, 0.28825656, 0.4501376, 0.5768339]",2.244898,"[1.0706645, -1.4817994, -1.3028871, 2.2972467]"
6,1.299309,"[0.34323, -0.21372998, 1.2347937, 5.329384]",-0.110468,5.279264,2.258104,"[-1.3809851, -1.7354782, -0.42429867, 2.4527736]",1.715109,"[-1.0010291, -1.3818703, -0.1731301, 1.8004836]",0.529219,"[-0.13340673, -0.4847811, -0.16511486, 0.5292192]",1.061704,"[-0.76368505, -0.7373468, 0.017914124, 1.0617044]",0.442212,"[-0.34188595, -0.2385023, -0.1475887, 0.46371502]",0.206799,"[-0.07449945, -0.16341971, -0.10251624, 0.2494...",2.835705,"[1.7242151, 1.5217482, 1.6590923, 2.8766105]"
7,2.282011,"[0.55363226, -0.4947052, 2.1578531, 5.50413]",-0.199456,5.211359,1.800868,"[-1.6871085, 0.60065913, -0.18973048, 2.042229]",1.054447,"[-0.98339355, 0.36952043, 0.090832196, 1.1882766]",0.368891,"[-0.32721296, 0.021299647, -0.16899335, 0.3688...",0.674366,"[-0.53784835, 0.31064147, 0.26266113, 0.67436